This notebook will demonstrate how to use Python and some of its modules to estimate parameters of a multi-variate normal distribution for a randomly-generated set of data.  It will use the maximum likelihood estimation (MLE) approach to find the parameters.

Created on August 29-30 and September 3-5, 2022 by Kevin Spradlin, Jr.

First, import some Python modules.

In [1]:
import typing
import math
import numpy as np
import scipy.stats as stats
import scipy.optimize as optimize

Next, create a set of random multi-variate normal variates using the scipy.stats ***multivariate_normal*** function.

In [2]:
def is_matrix_pos_semidef(test_matrix: np.array) -> typing.Dict:
    """
    This function will test the array test_matrix to see if it's positive 
     semi-definite, using this definition:

     "A positive semidefinite matrix is a Hermitian matrix all of whose eigenvalues are nonnegative."
     https://mathworld.wolfram.com/HermitianMatrix.html

     It will check if the matrix is square, Hermitian, and only has positive 
     eigenvalues.

     The function will return a dictionary.  One key is 'pass_test', which
      will be True if it's positive semi-definite and False if not.  The
      other key is 'message' will will state why the matrix failed the test,
      or will be blank it it passed the test.

    Created on June 8, 2022
    """

    pass_test: bool = False
    message: str = ''


    # check if the matrix is square
    matrix_dimensions: List = list(test_matrix.shape)

    if len(matrix_dimensions) != 2:
        message = 'Matrix needs to have 2 dimensions'
        return {'pass_test': pass_test, 'message': message}

    if matrix_dimensions[0] != matrix_dimensions[1]:
        message = 'Matrix needs to be square'
        return {'pass_test': pass_test, 'message': message}


    # check if the matrix is Hermitian
    complex_conjugate: np.array = np.matrix.getH(test_matrix)

    if not np.array_equal(test_matrix, complex_conjugate):
        message = "Matrix isn\'t Hermitian - equal to complex conjugate of itself"
        return {'pass_test': pass_test, 'message': message}


    # calculate the matrix's eigenvalues and check if any are negative
    for current_eigenvalue in np.linalg.eigvals(test_matrix):
        if current_eigenvalue < 0:
            message = f"Matrix has eigenvalue of {current_eigenvalue:6.4f}"
            break


    if message:
        return {'pass_test': pass_test, 'message': message}
    else:
        return {'pass_test': True, 'message': ''}

In [3]:
# let the user define the means and covariance matrix of the multi-variate normal distribution
mean_vector: np.ndarray = np.asarray([2.85, -3.65], dtype=np.float32)

# this covariance matrix doesn't lead to the optimization function landing on a matrix
#  that's not positive semi-definite.
#cov_matrix: np.ndarray = np.asarray([[7.05, -3.15], [-3.15, 2.45]], dtype=np.float32)

# this covariance matrix leads to the optimization function landing on a matrix that's
#  not positive semi-definite.  that's where the 'trial_pos_semidef_matrix' function
#  comes into play.
cov_matrix: np.ndarray = np.asarray([[7.05, -3.65], [-3.65, 2.25]], dtype=np.float32)

    
# check that the matrix is positive semi-definite
function_results: typing.Dict = is_matrix_pos_semidef(cov_matrix)

if not function_results['pass_test']:
    print(function_results['message'])


# generate 10,000 random variates
random_sample: np.ndarray = stats.multivariate_normal.rvs(mean=mean_vector, cov=cov_matrix, size=10000)


Before we use the MLE approach, let's calculate the basic statistics of the sample of random multi-variate normal variates.  They should show that the mean vector and standard deviations of the sample are close to the values you defined, since the parameters that maximize the likelihood function are the sample means and standard deviations.  The sample statistics should also show that the skewnesses and excess kurtosis (kurtoses ?) are close to zero.

In [4]:
results = stats.describe(random_sample)
print(f"Sample means: {np.array2string(results[2], precision=4):s}")
print(f"Sample variances: {np.array2string(results[3], precision=4):s}")
print(f"Sample skewnesses: {np.array2string(results[4], precision=4):s}")
print(f"Sample excess kurtosis: {np.array2string(results[5], precision=4):s}")

Sample means: [ 2.786  -3.6179]
Sample variances: [7.021  2.2483]
Sample skewnesses: [-0.0334  0.0129]
Sample excess kurtosis: [-0.006  -0.0288]


---

Now, we'll use the MLE approach.
First define a function that calculates the log-likelihood function.  Then find the parameters that maximize the log-likelihood function.

In [5]:
def decode_lh_parameters_norm(parameters: np.ndarray, num_dimensions: int) -> typing.Dict:
    """
    This function will convert the elements of the parameters array into a
     mean vector and a covariance matrix.  It will return them as values
     in a dictionary.
    
    Created on August 30 and September 5, 2022
    """

    mean_vector: np.ndarray = parameters[0:num_dimensions]

    cov_matrix: np.ndarray = np.zeros((num_dimensions, num_dimensions), dtype=np.float32)

    addend: int = num_dimensions
    for current_column in range(num_dimensions):
        addend -= current_column
        
        for current_row1 in range(0, current_column):
            cov_matrix[current_row1, current_column] = cov_matrix[current_column, current_row1]
        
        for current_row2 in range(current_column, num_dimensions):
            cov_matrix[current_row2, current_column] = \
              parameters[current_row2 + current_column * num_dimensions + addend]
    
    
    return {'mean_vector': mean_vector, 'covariance_matrix': cov_matrix}

In [6]:
def trial_pos_semidef_matrix(test_matrix: np.array) -> typing.Dict:
    """
    This function will take a matrix return a new matrix, based on the
     eigendecomposition of the original matrix.  The original matrix's 
     eigenvalues and vectors will be calculated.  If any eigenvalues are
     non-positive, then they will be set to be slightly positive.  The
     new matrix will be calculated from the original matrix's eigenvectors,
     any of the original eigenvalues that are positive, and the adjusted, 
     now-slightly positive eigenvalues.
     
    The new matrix will be positive semi-definite.
    
    Got information about eigendecomposition from this site:
    https://mathworld.wolfram.com/EigenDecomposition.html
    
    Created on September 3, 2022
    """    
    
    message: str = ''
    
    
    # first, make sure the matrix is symmetric
    matrix_dimensions: List = list(test_matrix.shape)

    if len(matrix_dimensions) != 2:
        message = 'Matrix needs to have 2 dimensions'
        return {'any_errors': True, 'message': message}

    if matrix_dimensions[0] != matrix_dimensions[1]:
        message = 'Matrix needs to be square'
        return {'any_errors': True, 'message': message}

 
    if not np.array_equal(test_matrix, test_matrix.T):
        message = "Matrix isn\'t symmetric"
        return {'any_errors': True, 'message': message}
   

    # next, calculate the eigenvalues and eigenvectors of the matrix
    matrix_p: np.ndarray = np.zeros((matrix_dimensions[0], matrix_dimensions[0]), dtype=np.float32)
    matrix_d: np.ndarray = np.zeros((matrix_dimensions[0], matrix_dimensions[0]), dtype=np.float32)
    
    test_eigenvalues, test_eigenvectors = np.linalg.eig(test_matrix)

    
    for current_index, current_eigenvalue in enumerate(test_eigenvalues):
        matrix_p[:,current_index] = test_eigenvectors[:,current_index]
       
        if current_eigenvalue >= 0.0:
            matrix_d[current_index, current_index] = current_eigenvalue
        else:
            matrix_d[current_index, current_index] = np.random.rand() / 10.0

    
    adjusted_matrix: np.ndarray = np.matmul(matrix_p, matrix_d)
    adjusted_matrix = np.matmul(adjusted_matrix, np.linalg.inv(matrix_p))
 

    #print(f"Eigenvalues: {np.array2string(test_eigenvalues, precision=4):s}")
    #print(f"Eigenvectors: {np.array2string(test_eigenvectors, precision=4):s}")
    
    #print(f"Eigenvector matrix: {np.array2string(matrix_p, precision=4):s}")
    #print(f"Eigenvalue matrix: {np.array2string(matrix_d, precision=4):s}")
    #print(f"Test matrix: {np.array2string(test_matrix, precision=4):s}")
    #print(f"Adjusted matrix: {np.array2string(adjusted_matrix, precision=4):s}")
   
    
    return {'any_errors': False, 'message': '', 'adjusted_matrix': adjusted_matrix}

In [7]:
def log_likelihood_multivariate_norm(parameters: np.ndarray, num_dimensions: int, variates: np.ndarray) -> float:
    """
    Returns the negative of the log-likelihood function.  The scipy.optimize module only
     has a 'minimize' function, so you need to use it to minimize the negative of the
     log-likelihood function, which will maximize the positive of the log-likelihood
     function.

    Expect parameters[0:dimension] to be the mean vector and parameters[dimension:] are the
     elements of the lower triangle of the covariance matrix.

    Created on August 29 and September 3, 2022
    """
    
    function_results: typing.Dict = decode_lh_parameters_norm(parameters, num_dimensions)

    mean_vector: np.ndarray = function_results['mean_vector']

    cov_matrix: np.ndarray = function_results['covariance_matrix']

#    print(f"Mean vector: {np.array2string(mean_vector, precision=4):s}")
#    print(f"Covariance matrix: {np.array2string(cov_matrix, precision=4):s}")
    
    
    # need to ensure that the covariance matrix is positive definite or semi-definite
    # first, test it.  if it passes the test, then calculate the likelihood function
    # if it fails the test, calculate an adjusted covariance matrix that passes the
    #  test and use it to calculate the likelihood function
    function_results: typing.Dict = is_matrix_pos_semidef(cov_matrix)

    if not function_results['pass_test']:
        other_function_results: typing.Dict = trial_pos_semidef_matrix(cov_matrix)
        
        if other_function_results['any_errors']:
            print(other_function_results['message'])
            return 0.0
        else:
            cov_matrix = other_function_results['adjusted_matrix']
    
    
    temp_sum: float = -np.sum(stats.multivariate_normal.logpdf(variates, 
                                                               mean=mean_vector, 
                                                               cov=cov_matrix, 
                                                               allow_singular=False))

    print(f"Log likelihood: {temp_sum:12.6f}")

    return temp_sum

In order to use the numpy ***minimize*** function, the parameters of the likelihood function need to be fed into the ***minimize*** function as a vector. <br><br>
To do this, I set up a ***init_parameters*** one-dimensional NumPy array.  The first part of the array has the elements in the mean vector and the remaining part of the array has the lower triangle of the covariance matrix. <br><br>
Suppose you want to estimate the parameters of multi-variate normal distribution with an n-dimensional mean vector and a n-by-n covariance matrix.  The first n elements of the ***init_parameters*** array can store the values of the mean vector. <br><br>
You need to ensure that the covariance matrix that's used by the likelihood function is symmetric.  If you simply put every element of the covariance matrix into the ***init_parameters*** array, then there's a good chance that the ***minimize*** function would set elements in the array so that the matrix would no longer be symmetric.  To ensure that this won't happen, I only save the elements of the lower triangle of the covariance matrix in the ***init_parameters*** array.  Inside of the ***log_likelihood_multivariate_norm*** function, I use the remaining elements in the array to set up a symmetric matrix.

Here's where I set up the ***init_parameters*** array.

In [8]:
num_dimensions: int = random_sample.shape[1]
num_elements: int = int(num_dimensions * (num_dimensions + 3) / 2)

print(f"Number of dimensions: {num_dimensions:d}")
print(f"Number of parameters in optimization problem: {num_elements:d}")
    
init_parameters: np.ndarray = np.zeros(num_elements, dtype=np.float32)

# set the starting values for the mean vector equal to the sample means.
sample_stats = stats.describe(random_sample)

for current_element in range(num_dimensions):
#    init_parameters[current_element] = 1.0
    init_parameters[current_element] = sample_stats[2][current_element]

# now set the starting values for the covariance matrix.  set the diagonal
#  elements equal to the same variances.
current_element:int = num_dimensions
for step_size in range(num_dimensions, 0, -1):
    if current_element < num_elements:
        init_parameters[current_element] = sample_stats[3][num_dimensions - step_size]
        current_element += step_size


print(np.array2string(init_parameters, precision=4))

Number of dimensions: 2
Number of parameters in optimization problem: 5
[ 2.786  -3.6179  7.021   0.      2.2483]


---

Finally, I run the ***minimize*** function to perform the MLE and obtain the estimated mean vector and covariance matrix.<br><br>


You also need to add some boundary conditions for the optimization function.  It's easier to set them up at the same time you set up the ***init_parameters*** array.  The conditions are:
* elements on the diagonal in the shape matrix need to be positive.<br><br>

You can see from the results that the values in the mean vector and covariance matrix are somewhat close to the values you originally entered.<br>

In [9]:
boundary_cond: typing.List = []

for index, value in enumerate(init_parameters):
    if index < num_dimensions:
        boundary_cond.append([None,None])
    else:
        if value == 1:
            boundary_cond.append([0,100000])
        else:
            boundary_cond.append([None,None])


opt_results = optimize.minimize(log_likelihood_multivariate_norm, 
                                x0=init_parameters, 
                                args=(num_dimensions, random_sample, ), 
                                method='Nelder-Mead', bounds=boundary_cond,
                                options={'maxiter': num_elements * 1000,
                                         'maxfev': num_elements * 1000})


function_results: typing.Dict = decode_lh_parameters_norm(opt_results['x'], num_dimensions)           

print(f"Estimated mean vector: {np.array2string(function_results['mean_vector'], precision=4):s}")
print(f"Estimated covariance matrix: {np.array2string(function_results['covariance_matrix'], precision=4):s}")
print(opt_results['message'])

print(f"\nActual mean vector: {np.array2string(mean_vector, precision=4):s}")
print(f"Actual covariance matrix: {np.array2string(cov_matrix, precision=4):s}")

Log likelihood: 42173.236657
Log likelihood: 42187.055210
Log likelihood: 42246.010331
Log likelihood: 42179.116054
Log likelihood: 42173.812546
Log likelihood: 42179.116048
Log likelihood: 42248.939185
Log likelihood: 42191.663252
Log likelihood: 42193.671646
Log likelihood: 42178.430791
Log likelihood: 42188.724989
Log likelihood: 42177.161199
Log likelihood: 42182.261889
Log likelihood: 42175.210026
Log likelihood: 42181.150600
Log likelihood: 42175.260321
Log likelihood: 42177.321296
Log likelihood: 42174.643419
Log likelihood: 42176.472272
Log likelihood: 42174.365406
Log likelihood: 42175.191075
Log likelihood: 42175.399972
Log likelihood: 42174.073900
Log likelihood: 42175.023780
Log likelihood: 42173.998559
Log likelihood: 42174.532098
Log likelihood: 42173.846436
Log likelihood: 42173.906075
Log likelihood: 42174.082609
Log likelihood: 42173.680696
Log likelihood: 42174.052178
Log likelihood: 42173.651192
Log likelihood: 42174.182012
Log likelihood: 42173.605151
Log likelihood

Log likelihood: 33080.547916
Log likelihood: 33082.143688
Log likelihood: 33080.501716
Log likelihood: 33082.904002
Log likelihood: 33080.389700
Log likelihood: 33080.936463
Log likelihood: 33082.020513
Log likelihood: 33080.340216
Log likelihood: 33081.371743
Log likelihood: 33080.386844
Log likelihood: 33080.582208
Log likelihood: 33080.290791
Log likelihood: 33080.464960
Log likelihood: 33080.347755
Log likelihood: 33080.596622
Log likelihood: 33080.240590
Log likelihood: 33080.908694
Log likelihood: 33080.194246
Log likelihood: 33080.351494
Log likelihood: 33080.224219
Log likelihood: 33080.369027
Log likelihood: 33080.222633
Log likelihood: 33080.448532
Log likelihood: 33080.213061
Log likelihood: 33080.220840
Log likelihood: 33080.271393
Log likelihood: 33080.186901
Log likelihood: 33080.218429
Log likelihood: 33080.205743
Log likelihood: 33080.244233
Log likelihood: 33080.179504
Log likelihood: 33080.184316
Log likelihood: 33080.213613
Log likelihood: 33080.174810
Log likelihood